In [ ]:
#  Clickhouse (via Tabix) was used to solve the tasks.

In [ ]:
# LESSON 1 -- Intro to SQL

In [ ]:
# Show the first 10 rows.
SELECT 
  * 
FROM 
  checks 
LIMIT 
  10

In [ ]:
# Show top-10 values in Rub column. Sort them in descending order.
SELECT 
  * 
FROM 
  checks 
ORDER BY 
  Rub DESC 
LIMIT 
  10;

In [ ]:
# Show the first 15 rows of the table renaming Rub to Revenue 
# and sort the result by UserID (ascending order).
SELECT 
  Rub AS Revenue, 
  BuyDate, 
  UserID 
FROM 
  checks 
ORDER BY 
  UserID ASC 
LIMIT 
  15

In [ ]:
# Check the dates range we have in the checks table.
SELECT 
  MIN(BuyDate) AS MinDate, 
  MAX(BuyDate) AS MaxDate 
FROM 
  checks

In [ ]:
# Show 10 unique UserID's. Sort the results by UserID in ascending order.
SELECT 
  DISTINCT UserID 
FROM 
  checks 
ORDER BY 
  UserID ASC 
LIMIT 
  10

In [ ]:
# Show the sales data: orders purchased on the 8th of March 2019.
# Sort the results by Rub in descending order. Limit the data to the first 10 rows.
SELECT 
  * 
FROM 
  checks 
WHERE 
  BuyDate IN ('2019-03-08') 
ORDER BY 
  Rub DESC 
LIMIT 
  10

In [ ]:
# Show unique UserID's for buyers made purchase on the 1st of September 2019
# and their expenditures were higer than 2000 rubles.
# Sort the resul by UserID in descending order.
SELECT 
  DISTINCT UserID 
FROM 
  checks 
WHERE 
  (
    BuyDate = '2019-09-01' 
    AND Rub > 2000
  ) 
ORDER BY 
  UserID DESC

In [ ]:
# LESSON 2 -- Grouping and Aggregate

In [ ]:
# Count number of purchases per each client, name the column as NumChecks.
# Sort the result by NumChecks in descending order. Limit output to 10 rows.
SELECT 
  UserID, 
  COUNT(Rub) AS NumChecks 
FROM 
  checks 
GROUP BY 
  UserID 
ORDER BY 
  NumChecks DESC 
LIMIT 
  10

In [ ]:
# Count sum of purchases made by each client, name the column as Revenue.
# Sort the result by Revenue in descending order. Limit output to 10 rows.
SELECT 
  UserID, 
  COUNT(Rub) AS NumChecks, 
  SUM(Rub) AS Revenue 
FROM 
  checks 
GROUP BY 
  UserID 
ORDER BY 
  Revenue DESC 
LIMIT 
  10

In [ ]:
# Count by date min, max and average money spent.
# Use sort by BuyDate in descending order. Limit output ot 10 rows.
SELECT 
  BuyDate, 
  MIN(Rub) AS MinCheck, 
  MAX(Rub) AS MaxCheck, 
  AVG(Rub) AS AvgCheck 
FROM 
  checks 
GROUP BY 
  BuyDate 
ORDER BY 
  BuyDate DESC 
LIMIT 
  10

In [ ]:
# Find customers who has spent more than 10000 rubles.
# Sort the result by UserID in descending order. Limit output to 10 rows.
SELECT 
  UserID, 
  SUM(Rub) AS Revenue 
FROM 
  checks 
GROUP BY 
  UserID 
HAVING 
  Revenue > 10000 
ORDER BY 
  UserID DESC 
LIMIT 
  10

In [ ]:
# Count Revenue by country. Sort the result by Revenue in descenig order.
# Limit output to 10 rows.
SELECT 
  Country, 
  SUM(Quantity * UnitPrice) AS Revenue 
FROM 
  default.retail 
GROUP BY 
  Country 
ORDER BY 
  Revenue DESC 
LIMIT 
  10

In [ ]:
# Count average number of purchases and an average price of a unit.
# Group the result by country and sort it by an average price of a unit 
# in descending order. Do not include purchases having "Manual" in Description.
SELECT 
  Country, 
  AVG(UnitPrice), 
  AVG(Quantity) 
FROM 
  default.retail 
WHERE 
  Description != 'Manual' 
GROUP BY 
  Country 
ORDER BY 
  AVG(UnitPrice) DESC

In [ ]:
# Calculate Revenue by month. Sort the result by Revenue in descending order.
# Do not include purchases having "Manual" in Description.
SELECT 
  toStartOfMonth(InvoiceDate) AS month, 
  SUM(UnitPrice * Quantity) AS revenue 
FROM 
  default.retail 
WHERE 
  Description != 'Manual' 
GROUP BY 
  month 
ORDER BY 
  revenue DESC

In [ ]:
# Calculate an average price of a unit bought in march 2011.
# Group the results by CustomerID and sort it by an average price of a unit in
# descending order. Do not include purchases having "Manual" in Description.
# Limit output to 10 rows.
SELECT 
  CustomerID, 
  AVG(UnitPrice) AS avg_purch_in_march 
FROM 
  default.retail 
WHERE 
  Description != 'Manual' 
GROUP BY 
  CustomerID 
HAVING 
  toStartOfMonth(InvoiceDate) = '2011-03-01' 
ORDER BY 
  avg_purch_in_march DESC 
LIMIT 
  10

In [ ]:
# Group the data by month and country and calculate average, min, max quanitity of
# bought items. Sort the result by average quantity in descending order.
# Do not include purchases having "Manual" in Description and purchases having a
# negative quantity. Show the results for United Kingdom only.
SELECT 
  Country, 
  toStartOfMonth(InvoiceDate) AS month, 
  AVG(Quantity) AS avg_quantity, 
  MIN(Quantity) AS min_quantity, 
  MAX(Quantity) AS max_quantity 
FROM 
  default.retail 
WHERE 
  Description != 'Manual' 
  AND Quantity > 0 
GROUP BY 
  Country, 
  month 
HAVING 
  Country = 'United Kingdom' 
ORDER BY 
  avg_quantity DESC

In [ ]:
# LESSON 3 -- Joins

In [ ]:
# Join two tables on DeviceID. Sort the result by DeviceID in descenig order.
# Limit output to 100 rows.
SELECT 
  l.AppPlatform AS AppPlatform, 
  l.events AS events, 
  l.EventDate AS EventDate, 
  r.UserID AS UserID 
FROM 
  events AS l 
  LEFT JOIN devices AS r ON l.DeviceID = r.DeviceID 
ORDER BY 
  DeviceID DESC 
LIMIT 
  100

In [ ]:
# Join three tables to find out the Source that brough users with the most amount 
# of money spent. Sort the result by money spent in descenig order. 
# Limit output to 100 rows.
SELECT 
  l.Source, 
  SUM(r.Rub) AS Rub 
FROM 
  (
    SELECT 
      l.Source, 
      l.DeviceID, 
      r.UserID 
    FROM 
      installs AS l 
      JOIN devices AS r ON l.DeviceID = r.DeviceID
  ) AS l 
  JOIN checks AS r ON l.UserID = r.UserID 
GROUP BY 
  l.Source 
ORDER BY 
  Rub DESC 
LIMIT 
  100

In [ ]:
# Find the number of unique users who bought something in our app.
# Group and sort the result by Source. Limit output to 100 rows.
SELECT 
  l.Source, 
  uniqExact(r.UserID) 
FROM 
  (
    SELECT 
      l.Source, 
      l.DeviceID, 
      r.UserID 
    FROM 
      installs AS l 
      JOIN devices AS r ON l.DeviceID = r.DeviceID
  ) AS l 
  JOIN checks AS r ON l.UserID = r.UserID 
GROUP BY 
  l.Source 
ORDER BY 
  l.Source DESC 
LIMIT 
  100

In [ ]:
# Calculate Revenue, min, max, average check.
# Group and sort the result by Source. Limit output to 100 rows.
SELECT 
  l.Source, 
  SUM(r.Rub) AS Revenue, 
  MIN(r.Rub) AS min_check, 
  MAX(r.Rub) AS max_check, 
  AVG(r.Rub) AS avg_check 
FROM 
  (
    SELECT 
      l.Source, 
      l.DeviceID, 
      r.UserID 
    FROM 
      installs AS l 
      JOIN devices AS r ON l.DeviceID = r.DeviceID
  ) AS l 
  JOIN checks AS r ON l.UserID = r.UserID 
GROUP BY 
  l.Source 
ORDER BY 
  l.Source DESC 
LIMIT 
  100

In [ ]:
# Show DeviceID's of users who has made at least one purchase over the last month
# (October 2019). Sort the result by DeviceID in ascending order.
# Limit output to 100 rows.
SELECT 
  r.BuyDate, 
  l.DeviceID, 
  SUM(r.Rub) AS total 
FROM 
  (
    SELECT 
      l.Source, 
      l.DeviceID, 
      r.UserID 
    FROM 
      installs AS l 
      JOIN devices AS r ON l.DeviceID = r.DeviceID
  ) AS l 
  JOIN checks AS r ON l.UserID = r.UserID 
GROUP BY 
  l.DeviceID, 
  r.BuyDate 
HAVING 
  total > 0 
  AND toStartOfMonth(
    CAST(r.BuyDate AS Date)
  ) = '2019-10-01' 
ORDER BY 
  l.DeviceID ASC 
LIMIT 
  100

In [ ]:
# Check an average amount of views (events) by users from various Platforms and
# Sources. Sort the result by an average amount of views for each platform-source pair
# in descending order. Limit output to 100 rows.
SELECT 
  r.Platform, 
  r.Source, 
  AVG(l.events) AS avg_views 
FROM 
  events AS l 
  JOIN installs AS r ON l.DeviceID = r.DeviceID 
GROUP BY 
  r.Platform, 
  r.Source 
ORDER BY 
  avg_views DESC 
LIMIT 
  100

In [ ]:
# Count the number of unique DeviceIDs in installs only for those DeviceIDs that
# have views (events) in events table for Platform Android.
SELECT 
  COUNT(DISTINCT r.DeviceID) 
FROM 
  events AS l 
  INNER JOIN installs AS r ON l.DeviceID = r.DeviceID 
WHERE 
  r.Platform = 'android' 
LIMIT 
  100

In [ ]:
# Calculate the conversion rate from installs to views for users using iOS platform.
SELECT 
  (
    COUNT(DISTINCT r.DeviceID) / COUNT(DISTINCT l.DeviceID)
  ) AS conversion 
FROM 
  installs AS l FULL 
  JOIN events AS r ON l.DeviceID = r.DeviceID 
WHERE 
  l.Platform = 'iOS' 

In [ ]:
# Print 10 unique DeviceIDs that are presented in the table events but are missing from the table
# installs because of a logging mistake. List them by DeviceIDs in descending order.
  DISTINCT(r.DeviceID) 
FROM 
  installs AS l RIGHT ANTI 
  JOIN events AS r ON l.DeviceID = r.DeviceID 
ORDER BY 
  r.DeviceID DESC 
LIMIT 
  10

In [ ]:
# LESSON 4 -- Data Types

In [ ]:
# Find the month and the year with the most number of new hosts registartions
SELECT 
  COUNT(DISTINCT host_id) AS hosts_num, 
  toStartOfMonth(
    toDateOrNull(host_since)
  ) AS start_date 
FROM 
  default.listings 
WHERE 
  start_date IS NOT NULL 
GROUP BY 
  start_date 
ORDER BY 
  hosts_num DESC 
Limit 
  100

In [ ]:
# Find an average response rate for hosts and superhosts

# for superhosts
SELECT 
  AVG(sup.freq) AS avg_response_super 
FROM 
  (
    SELECT 
      DISTINCT host_id, 
      toInt8OrNull(
        replaceAll(host_response_rate, '%', '')
      ) AS freq, 
      host_is_superhost 
    FROM 
      default.listings 
    WHERE 
      host_is_superhost == 't' 
      AND freq IS NOT NULL
  ) AS sup

# for regular hosts
SELECT 
  AVG(not_sup.freq) AS avg_response_regular 
FROM 
  (
    SELECT 
      DISTINCT host_id, 
      toInt8OrNull(
        replaceAll(host_response_rate, '%', '')
      ) AS freq, 
      host_is_superhost 
    FROM 
      default.listings 
    WHERE 
      host_is_superhost == 'f' 
      AND freq IS NOT NULL
  ) AS not_sup

In [ ]:
# Alternative solution using only one query
SELECT 
  host_is_superhost, 
  AVG(
    toInt8OrNull(
      replaceAll(host_response_rate, '%', '')
    )
  ) AS avg_response_rate 
FROM 
  (
    SELECT 
      host_is_superhost, 
      host_response_rate, 
      host_id 
    FROM 
      default.listings 
    GROUP BY 
      host_is_superhost, 
      host_response_rate, 
      host_id
  ) 
GROUP BY 
  host_is_superhost

In [ ]:
# Assign property ids to arrays and list them grouped by host_id.
# Caclculate an average price of a property for a host_id. 
# Find the host_id with the highest average price. 
SELECT 
  DISTINCT host_id, 
  AVG(
    toDecimal32OrNull(
      replaceRegexpAll(price, '[$,]', ''), 
      2
    )
  ) AS avg_price, 
  groupArray(id) 
FROM 
  default.listings 
GROUP BY 
  host_id 
ORDER BY 
  avg_price DESC, 
  host_id DESC 
LIMIT 
  100

In [ ]:
# Calculate the difference between the min and max price for each host_id
SELECT 
  DISTINCT host_id, 
  MIN(
    toDecimal32OrNull(
      replaceRegexpAll(price, '[$,]', ''), 
      2
    )
  ) AS min_price, 
  MAX(
    toDecimal32OrNull(
      replaceRegexpAll(price, '[$,]', ''), 
      2
    )
  ) AS max_price, 
  max_price - min_price AS diff 
FROM 
  default.listings 
GROUP BY 
  host_id 
HAVING 
  diff > 0 
ORDER BY 
  diff DESC, 
  host_id DESC 
LIMIT 
  100

In [ ]:
# Now group the data by a room_type.
# Calculate an average price, average security deposit and an average cleaning fee.
SELECT 
  room_type, 
  AVG(
    toDecimal32OrNull(
      replaceRegexpAll(price, '[$,]', ''), 
      2
    )
  ) AS avg_price, 
  AVG(
    toDecimal32OrNull(
      replaceRegexpAll(security_deposit, '[$,]', ''), 
      2
    )
  ) AS avg_deposit, 
  AVG(
    toDecimal32OrNull(
      replaceRegexpAll(cleaning_fee, '[$,]', ''), 
      2
    )
  ) AS avg_cleaning 
FROM 
  default.listings 
GROUP BY 
  room_type 
ORDER BY 
  avg_deposit DESC

In [ ]:
# What districts do have the lowest average price per night?
SELECT 
  neighbourhood_cleansed, 
  AVG(
    toDecimal32OrNull(
      replaceRegexpAll(price, '[$,]', ''), 
      2
    )
  ) AS avg_price 
FROM 
  default.listings 
GROUP BY 
  neighbourhood_cleansed 
ORDER BY 
  avg_price ASC 
LIMIT 
  100

In [ ]:
# What districts do have the largest average square of property for a room type
# 'Entire home/apt'. Sort by average squre and list the top-3.
SELECT 
  neighbourhood_cleansed, 
  AVG(
    toDecimal32OrNull(square_feet, 2)
  ) AS avg_square 
FROM 
  default.listings 
WHERE 
  room_type == 'Entire home/apt' 
GROUP BY 
  neighbourhood_cleansed 
ORDER BY 
  avg_square DESC 
LIMIT 
  3

In [ ]:
# Find the 'Private room' having the closest location to the 
# city center (13.4050, 52.5200)
SELECT 
  id, 
  toFloat64OrNull(latitude) AS lat, 
  toFloat64OrNull(longitude) AS lng, 
  geoDistance(13.4050, 52.5200, lng, lat) AS dist 
FROM 
  default.listings 
WHERE 
  room_type == 'Private room' 
ORDER BY 
  dist ASC 
Limit 
  100

In [ ]:
# LESSON 5 -- WITH, VIEWS and Creating Tables

In [ ]:
# PART 1

In [ ]:
# Show advertisement where review score is above average and 
# the number of reviews per month is less than three.
# Sort the result in descending order by number of reviews per month and 
# by review score.
WITH (
  SELECT 
    AVG(
      toDecimal32OrNull(review_scores_rating, 2)
    ) 
  FROM 
    listings 
  WHERE 
    toDecimal32OrNull(review_scores_rating, 2) IS NOT NULL
) AS avg_scores_rating


SELECT 
  id, 
  toDecimal32OrNull(review_scores_rating, 2) AS scores_rating, 
  avg_scores_rating, 
  toDecimal32(reviews_per_month, 2) AS review_per_month 
FROM 
  listings 
WHERE 
  scores_rating > avg_scores_rating 
  AND review_per_month < 3 
ORDER BY 
  review_per_month DESC, 
  scores_rating DESC 
LIMIT 
  100

In [ ]:
# Calculate an average distance to the city center and show host_ids
# where property type is 'Private room' and the distance to the city center
# is less than an average distance to the city center.
# Pick a room that is further from the city center but is located closer 
# than an average distance to the city center.
WITH (
  SELECT 
    AVG(
      geoDistance(
        13.4050, 
        52.5200, 
        toFloat64OrNull(longitude), 
        toFloat64OrNull(latitude)
      )
    ) 
  FROM 
    listings 
  WHERE 
    room_type == 'Private room'
) AS avg_dist 


SELECT 
  host_id, 
  toFloat64OrNull(latitude) AS lat, 
  toFloat64OrNull(longitude) AS lng, 
  geoDistance(13.4050, 52.5200, lng, lat) AS dist, 
  avg_dist 
FROM 
  default.listings 
WHERE 
  room_type == 'Private room' 
  AND dist < avg_dist 
ORDER BY 
  dist DESC 
Limit 
  100

In [ ]:
# Query advertisements where distance to the city center is less than
# an averaga distance to the city center, price per night including a 
# cleaning fee is less than 100 USD, having the last review date not later
# than '2018-09-01' and have wifi in the list of amenities.
# Sort the result by review_scores_rating in descending order.
WITH (
  SELECT 
    AVG(
      geoDistance(
        13.4050, 
        52.5200, 
        toFloat64OrNull(longitude), 
        toFloat64OrNull(latitude)
      )
    ) 
  FROM 
    listings
) AS avg_dist 


SELECT 
  host_id, 
  toDecimal32OrNull(
    replaceRegexpAll(price, '[$,]', ''), 
    2
  ) * 7 + toDecimal32OrNull(
    replaceRegexpAll(cleaning_fee, '[$,]', ''), 
    2
  ) AS weekly_price, 
  cleaning_fee, 
  price, 
  weekly_price / 7 AS daily_price, 
  geoDistance(
    13.4050, 
    52.5200, 
    toFloat64OrNull(longitude), 
    toFloat64OrNull(latitude)
  ) AS dist, 
  avg_dist, 
  toFloat32OrNull(review_scores_rating) AS scores_rating, 
  last_review, 
  amenities 
FROM 
  default.listings 
WHERE 
  (
    dist < avg_dist 
    AND multiSearchAnyCaseInsensitive(amenities, [ 'wifi' ]) != 0
  ) 
  AND (
    daily_price < 100 
    AND last_review > '2018-08-31'
  ) 
ORDER BY 
  scores_rating DESC 
LIMIT 
  100

In [ ]:
# Pick available advertisements having number of reviews from unique users 
# above average. Sort the result by listing_id in ascending order.
WITH (
  SELECT 
    AVG(reviewer_id) 
  FROM 
    (
      SELECT 
        COUNT(
          DISTINCT(reviewer_id)
        ) AS reviewer_id, 
        listing_id 
      FROM 
        reviews 
      GROUP BY 
        listing_id
    )
) AS avg_uniq_revs 


SELECT 
  reviewer_id, 
  listing_id 
FROM 
  (
    SELECT 
      COUNT(
        DISTINCT(reviewer_id)
      ) AS reviewer_id, 
      listing_id 
    FROM 
      reviews 
    GROUP BY 
      listing_id
  ) AS a 
  JOIN (
    SELECT 
      listing_id 
    FROM 
      calendar_summary 
    WHERE 
      available == 't' 
    GROUP BY 
      listing_id
  ) AS b ON a.listing_id = b.listing_id 
WHERE 
  reviewer_id > avg_uniq_revs 
ORDER BY 
  listing_id ASC 
LIMIT 
  100

In [ ]:
# Split customers by four segments.
SELECT 
  CASE WHEN AVG(Rub) < 5 THEN 'A' WHEN AVG(Rub) >= 5 
  AND AVG(Rub) < 10 THEN 'B' WHEN AVG(Rub) >= 10 
  AND AVG(Rub) < 20 THEN 'C' ELSE 'D' END AS level, 
  UserID, 
  AVG(Rub) AS avg_rub 
FROM 
  checks 
GROUP BY 
  UserID 
ORDER BY 
  UserID ASC 
LIMIT 
  4

In [ ]:
# Find a number of customers for each segment and calculate
# the revenue for each segment. Sort the result by revenue in a
# descending order.
SELECT 
  COUNT(DISTINCT UserID) AS num_of_clients, 
  level, 
  SUM(sum_rub) AS revenue 
FROM 
  (
    SELECT 
      CASE WHEN AVG(Rub) < 5 THEN 'A' WHEN AVG(Rub) >= 5 
      AND AVG(Rub) < 10 THEN 'B' WHEN AVG(Rub) >= 10 
      AND AVG(Rub) < 20 THEN 'C' ELSE 'D' END AS level, 
      UserID, 
      AVG(Rub) AS avg_rub, 
      SUM(Rub) AS sum_rub 
    FROM 
      checks 
    GROUP BY 
      UserID 
    ORDER BY 
      UserID ASC
  ) 
GROUP BY 
  level 
ORDER BY 
  revenue DESC

In [ ]:
# If the listed property has a kitchen and a flexible cancellation policy
# rate it as 'good'; if it has only kithcen rate it as 'ok'; if it has neither 
# a kithcen nor a flexible cancallation policy rate it as 'not ok'.
# Sort the result by the new column with your rating.
SELECT 
  host_id, 
  CASE WHEN multiSearchAnyCaseInsensitive(amenities, [ 'kitchen' ]) != 0 
  AND multiSearchAnyCaseInsensitive(
    cancellation_policy, [ 'flexible' ]
  ) != 0 THEN 'good' WHEN multiSearchAnyCaseInsensitive(amenities, [ 'kitchen' ]) != 0 
  AND multiSearchAnyCaseInsensitive(
    cancellation_policy, [ 'flexible' ]
  ) == 0 THEN 'ok' ELSE 'not ok' END my_group 
FROM 
  default.listings 
ORDER BY 
  my_group ASC 
LIMIT 
  5

In [ ]:
# PART 2

In [ ]:
# Create a new table having columns with a proper data type.
CREATE TABLE test.reviews (
  listing_id UInt32, 
  id UInt32, 
  date DateTime('Europe/Moscow'), 
  reviewer_id UInt32, 
  reviewer_name String, 
  comments String
) ENGINE = MergeTree 
ORDER BY 
  listing_id, 
  id

In [ ]:
# Change the data type for date from DateTime to Date.
ALTER TABLE 
  test.reviews MODIFY COLUMN date Date

In [ ]:
# Delete rows with empty comments from reviews table.
ALTER TABLE 
  test.reviews DELETE 
WHERE 
  comments == ''

In [ ]:
# Create a view having reviewer_id and a number of ids from a reviews table.
CREATE VIEW test.reviews AS (
  SELECT 
    reviewer_id, 
    COUNT(id) AS reviews_count, 
  FROM 
    test.reviews 
  GROUP BY 
    reviewer_id
)

In [ ]:
# Create a new column reviewer_score (UInt8) in a reviews table 
# right after a reviewer_name column.
ALTER TABLE 
  test.reviews 
ADD 
  COLUMN reviewer_score UInt8 
AFTER 
  reviewer_name

In [ ]:
# Create a new column price (Float32) in a reviews table 
# right after a comments column.
ALTER TABLE 
  test.reviews 
ADD 
  COLUMN price Float32 
AFTER 
  comments

In [ ]:
# Update the price column by doubling its values for all lines
# having date > '2019-01-01'
ALTER TABLE 
  test.reviews 
UPDATE 
  price = price * 2 
WHERE 
  date > '2019-01-01'

In [ ]:
# LESSON 6 -- ETL (Python + SQL)

In [ ]:
tbc..